# Q&A Part 3: アーキテクチャ（Position Encoding, FFN, Encoder, Decoder）

Transformer学習中の質問と回答集

- Q17-Q18: Position Encoding
- Q19-Q20: Feed Forward Network
- Q21-Q23: 残差接続（Residual Connection）
- Q24: Encoder出力とCross-Attention
- Q25: Encoder Layerの直列接続
- Q26-Q27: 文脈を考慮した表現
- Q28: 特徴量ベクトルから単語への変換
- Q29-Q30: 翻訳タスクとTeacher Forcing
- Q31: シーケンス長の違いとCross-Attention

---

## Q17: Position Encodingは特徴量ベクトルの何次元を使うのか？

### 質問
Position Encodingは特徴量ベクトルの中で数次元を使うのですか？1次元だと三角関数なので、同じ位置を一意に決定できないと思います。

### 回答

**d_model次元すべてを使います。** 1次元では周期的なので位置を一意に特定できませんが、**複数の異なる周波数を組み合わせる**ことで一意に特定できます。

#### 数式の確認

```
PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
```

- `i = 0, 1, 2, ..., d_model/2 - 1`
- 各iに対してsin/cosのペアがある
- つまり **d_model次元すべて** がPosition Encodingに使われる

#### なぜ複数次元で一意になるのか？

**二進数との類似**で理解できます：

| 二進数 | Position Encoding |
|--------|-------------------|
| 各ビットは0か1の2値 | 各次元は-1〜1のsin/cos |
| 低位ビット: 速く変化 | 低次元(i小): 高周波 |
| 高位ビット: 遅く変化 | 高次元(i大): 低周波 |
| 複数ビットで多くの数を表現 | 複数次元で多くの位置を表現 |

#### 具体例（d_model=64の場合）

```
次元0,1 (i=0):  周期 ≈ 6.28   (高周波、近い位置を区別)
次元16,17 (i=8): 周期 ≈ 158   (中周波)
次元32,33 (i=16): 周期 ≈ 3969  (低周波)
次元62,63 (i=31): 周期 ≈ 62832 (最低周波、遠い位置を区別)
```

各次元が異なる「粒度」で位置を見ている：
- **低次元**: 隣り合う位置の違いを捉える（細かい目盛り）
- **高次元**: 大きく離れた位置の違いを捉える（粗い目盛り）

#### 視覚的理解

```
位置0:  [sin(0), cos(0), sin(0), cos(0), ...]
位置1:  [sin(1), cos(1), sin(0.1), cos(0.1), ...]  ← 各次元で異なる変化
位置2:  [sin(2), cos(2), sin(0.2), cos(0.2), ...]

        ↓高周波次元     ↓低周波次元
        大きく変化      ゆっくり変化
```

**64次元の組み合わせパターン**は膨大なので、実用上十分な数の位置を一意に表現できます。

#### コードで確認

In [ ]:
# Position Encodingが全次元を使うことを確認
import torch
import math

d_model = 64
max_len = 100

# Position Encodingを計算
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

pe[:, 0::2] = torch.sin(position * div_term)  # 偶数次元: sin
pe[:, 1::2] = torch.cos(position * div_term)  # 奇数次元: cos

print(f"Position Encoding shape: {pe.shape}")
print(f"→ 各位置に対して{d_model}次元すべてを使用\n")

# 各次元ペアの周期を確認
print("各次元ペアの周期（position単位）:")
print("-" * 40)
for i in [0, 8, 16, 24, 31]:
    period = 2 * math.pi / div_term[i].item()
    print(f"次元 {2*i:2d},{2*i+1:2d} (i={i:2d}): 周期 ≈ {period:8.1f}")

print("\n→ 異なる周波数の組み合わせで位置を一意に特定")

In [ ]:
# 各位置のPEが一意であることを確認
# 異なる位置のPE間のユークリッド距離を計算

print("位置間のユークリッド距離:")
print("-" * 40)
for pos1, pos2 in [(0, 1), (0, 10), (0, 50), (10, 11), (50, 51)]:
    distance = (pe[pos1] - pe[pos2]).norm().item()
    print(f"位置 {pos1:2d} と 位置 {pos2:2d}: 距離 = {distance:.4f}")

print("\n→ すべての位置ペアで距離 > 0（一意に区別可能）")

# 同じ位置は距離0
same_pos_distance = (pe[5] - pe[5]).norm().item()
print(f"\n位置 5 と 位置 5: 距離 = {same_pos_distance:.10f}（同一）")

#### まとめ

**Q: Position Encodingは特徴量ベクトルの何次元を使うのか？**

**A: d_model次元すべてを使います。**

##### ポイント

1. **1次元では不十分**: sin(pos)だけでは周期的で、位置0と位置2πが区別できない

2. **複数周波数の組み合わせ**: 各次元ペアが異なる周波数を持ち、組み合わせで一意に

3. **二進数に似た構造**:
   - 低次元 = 低位ビット（速く変化、近い位置を区別）
   - 高次元 = 高位ビット（遅く変化、遠い位置を区別）

4. **埋め込みベクトルへの加算**: Position Encodingはトークン埋め込みに**加算**される
   ```
   入力 = トークン埋め込み(d_model次元) + Position Encoding(d_model次元)
   ```

##### 注意
Position Encodingは「数次元を専用に使う」のではなく、**全次元に位置情報を重畳**します。トークンの意味情報と位置情報が同じ空間で混ざり合い、Attentionで両方を活用できます。

---

## Q18: Position Encodingは特徴量ベクトルの全要素に加法的に加わるのか？

### 質問
特徴量ベクトルの全ての要素に加法的に位置情報が加わるということですか？

### 回答

**はい、その通りです。** 各要素ごとに加算されます。

#### 具体的な計算

```python
入力 = トークン埋め込み + Position Encoding

# 各要素ごとの加算（element-wise addition）
[e₀, e₁, e₂, ..., e₆₃]      # トークン埋め込み（意味情報）
         +
[pe₀, pe₁, pe₂, ..., pe₆₃]  # Position Encoding（位置情報）
         ↓
[e₀+pe₀, e₁+pe₁, e₂+pe₂, ..., e₆₃+pe₆₃]  # 最終的な入力
```

#### なぜ加算で問題ないのか？

意味情報と位置情報が混ざっても問題にならない理由：

| 懸念 | 解決策 |
|------|--------|
| 情報が混ざって区別できない？ | Attentionが学習を通じて分離を学ぶ |
| 情報が上書きされない？ | 高次元空間（64〜512次元）は十分広い |
| 連結(concat)の方が良い？ | 加算の方がパラメータ効率的 |

#### 加算 vs 連結の比較

```
【連結（concat）の場合】
トークン埋め込み: [e₀, e₁, ..., e₆₃]        (64次元)
Position Encoding: [pe₀, pe₁, ..., pe₆₃]    (64次元)
結果: [e₀, ..., e₆₃, pe₀, ..., pe₆₃]        (128次元) ← 次元が2倍に！

【加算（add）の場合】
結果: [e₀+pe₀, e₁+pe₁, ..., e₆₃+pe₆₃]       (64次元) ← 次元そのまま
```

加算を使うことで：
- モデルサイズを抑えられる
- 計算コストが低い
- 実験的にも性能は十分

#### コードで確認

In [ ]:
# 加法的に位置情報が加わることを確認
import torch
import math

d_model = 8  # 見やすくするため小さい次元で
seq_len = 3

# トークン埋め込み（ダミー）
token_embedding = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0],  # 位置0のトークン
    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],  # 位置1のトークン
    [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],  # 位置2のトークン
])

# Position Encodingを計算
pe = torch.zeros(seq_len, d_model)
position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

# 加算
input_with_pe = token_embedding + pe

print("トークン埋め込み（位置0）:")
print(token_embedding[0].numpy())
print("\nPosition Encoding（位置0）:")
print(pe[0].numpy())
print("\n加算後（位置0）:")
print(input_with_pe[0].numpy())
print("\n確認: 各要素が element-wise に加算されている")
print(f"  {token_embedding[0, 0]:.2f} + {pe[0, 0]:.2f} = {input_with_pe[0, 0]:.2f}")

#### まとめ

**Q: Position Encodingは特徴量ベクトルの全要素に加法的に加わるのか？**

**A: はい、全要素に対してelement-wiseに加算されます。**

##### 重要なポイント

1. **加算（+）演算**: `入力 = embedding + position_encoding`

2. **次元は変わらない**: d_model次元のまま

3. **情報の混合は問題ない**: 
   - 高次元空間には十分な余裕がある
   - Attentionが意味と位置を使い分けることを学習する

4. **実装上のシンプルさ**: 
   ```python
   x = x + self.pe[:, :seq_len, :]  # これだけ！
   ```

##### なぜ連結(concat)ではなく加算なのか

- **効率性**: 次元数が2倍にならない
- **実験結果**: 加算でも十分な性能
- **論文の選択**: 元のTransformer論文で加算が採用された

---

## Q19: FFNの数式は「フィードフォワード」を示しているのか？

### 質問
FFNの数式はフィードフォワードを示していますか？バイアス付きの線形変換にしか見えませんが。

### 回答

**その観察は正しいです。** FFNの中身は「2つの線形変換 + 活性化関数」であり、特別なものではありません。

#### 数式を分解する

$$\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2$$

これは：
1. **線形変換1**: $xW_1 + b_1$
2. **活性化関数**: $\max(0, \cdot)$（ReLU）
3. **線形変換2**: $(\cdot)W_2 + b_2$

つまり、**2層のMLP（多層パーセプトロン）** そのものです。

#### 「フィードフォワード」の意味

「Feed Forward」はネットワークの**構造**を表す用語で、**情報の流れ方**を指しています：

| 用語 | 意味 | 例 |
|------|------|-----|
| **Feed Forward** | 入力→出力へ一方向に流れる | MLP, CNN |
| **Recurrent** | 出力が入力に戻る（ループ） | RNN, LSTM |
| **Feedback** | 後の層から前の層へ情報が戻る | 一部の特殊なネット |

```
Feed Forward（順伝播）:
入力 → 層1 → 層2 → 出力
      →→→→→→→→→→→
      （一方向のみ）

Recurrent（再帰）:
入力 → 層 → 出力
       ↑_____|
      （ループあり）
```

#### なぜ「FFN」と呼ぶのか

Transformerの文脈では、**Attentionと区別するため**にこの名前が使われています：

| コンポーネント | 特徴 |
|---------------|------|
| **Attention** | トークン間で情報を交換（相互作用あり） |
| **FFN** | 各トークンを独立に変換（相互作用なし） |

```
Attention:
トークン1 ←→ トークン2 ←→ トークン3
    （トークン間で情報交換）

FFN:
トークン1 → 変換 → 出力1
トークン2 → 変換 → 出力2  （同じ変換を独立に適用）
トークン3 → 変換 → 出力3
```

#### 本質的には何か

| 別名 | 説明 |
|------|------|
| MLP | Multi-Layer Perceptron（多層パーセプトロン） |
| 全結合層 | Fully Connected Layer |
| Dense層 | Kerasでの呼び方 |
| Position-wise FFN | Transformerでの正式名称（位置ごとのFFN） |

すべて同じものを指しています。

#### まとめ

**Q: FFNの数式は「フィードフォワード」を示しているのか？**

**A: はい。ただし「フィードフォワード」は中身ではなく、情報の流れ方（一方向）を表す用語です。**

- 中身は「2層MLP」（線形→活性化→線形）
- 「フィードフォワード」= 再帰やフィードバックがない構造
- Transformerでは「Attentionと対比する」ためにこの名前を使用
- 各トークンに独立に適用されるため「Position-wise FFN」とも呼ばれる

---

## Q20: 「1層あたりのパラメータ配分」の「1層」とは？

### 質問
1層あたりのパラメータ配分の「1層」とはどういうことですか？

### 回答

**Transformerにおける「1層」とは、Attention + FFN のセット**を指します。

#### Transformerの構造

```
┌─────────────────────────────────────┐
│           Transformer               │
│                                     │
│  ┌─────────────┐  ┌─────────────┐  │
│  │   Encoder   │  │   Decoder   │  │
│  │             │  │             │  │
│  │  ┌───────┐  │  │  ┌───────┐  │  │
│  │  │ 層 6  │  │  │  │ 層 6  │  │  │
│  │  ├───────┤  │  │  ├───────┤  │  │
│  │  │ 層 5  │  │  │  │ 層 5  │  │  │
│  │  ├───────┤  │  │  ├───────┤  │  │
│  │  │  ...  │  │  │  │  ...  │  │  │
│  │  ├───────┤  │  │  ├───────┤  │  │
│  │  │ 層 1  │  │  │  │ 層 1  │  │  │
│  │  └───────┘  │  │  └───────┘  │  │
│  └─────────────┘  └─────────────┘  │
└─────────────────────────────────────┘
```

元の論文では：
- Encoder: 6層
- Decoder: 6層

#### 「1層」の中身（Encoder層の場合）

```
┌────────────────────────────────┐
│          Encoder 1層           │
│                                │
│  入力                          │
│    ↓                          │
│  ┌──────────────────────────┐ │
│  │  Multi-Head Attention    │ │  ← パラメータの約1/3
│  └──────────────────────────┘ │
│    ↓                          │
│  ┌──────────────────────────┐ │
│  │  Feed Forward Network    │ │  ← パラメータの約2/3
│  └──────────────────────────┘ │
│    ↓                          │
│  出力                          │
└────────────────────────────────┘
```

（※ 実際にはLayer NormalizationとResidual Connectionも含まれます）

#### パラメータ数の内訳

d_model = 512, d_ff = 2048, num_heads = 8 の場合：

| コンポーネント | パラメータ数 | 割合 |
|---------------|-------------|------|
| Multi-Head Attention | 約100万 | ~33% |
| Feed Forward Network | 約210万 | ~67% |
| **1層の合計** | **約310万** | 100% |

#### 層を積み重ねる理由

```
入力 → 層1 → 層2 → 層3 → ... → 層N → 出力
```

- 各層で少しずつ抽象的な表現を獲得
- 浅い層: 表面的なパターン（文法など）
- 深い層: 意味的な理解（文脈など）
- 層を増やすほど表現力が上がる（ただし計算コストも増加）

#### 具体例

| モデル | 層数 | d_model | パラメータ総数 |
|--------|------|---------|---------------|
| Transformer (論文) | 6層 | 512 | ~65M |
| BERT-base | 12層 | 768 | ~110M |
| GPT-2 | 12層 | 768 | ~117M |
| GPT-3 | 96層 | 12288 | ~175B |

#### まとめ

**Q: 「1層あたりのパラメータ配分」の「1層」とは？**

**A: Attention + FFN のセット（= 1つのEncoder/Decoderブロック）を指します。**

- Transformerは複数の「層」を積み重ねた構造
- 各層の中に Attention と FFN が含まれる
- 層を増やすほどモデルの表現力が向上

# Transformer学習 Q&A

このノートブックは、Transformerの実装学習中に出た質問と回答をまとめたものです。

各Q&Aには、質問内容、回答、実際に動かせるコード例が含まれています。

---

## Q21: Add & Norm の「Add」とは何か？

**質問日**: 2025年12月21日

### 質問

Encoderの構造で「Add & Norm」という表記がありますが、「Add」の部分は何をしているのですか？

### 回答

「Add」は**残差接続（Residual Connection）**のことです。

#### 構造

```
        入力 x
          │
          ├──────────────┐
          ↓              │
    ┌──────────┐         │
    │ Sublayer │         │  ← Attention または FFN
    └────┬─────┘         │
         ↓               │
       F(x)              │
         │               │
         ↓               │
        (+) ←────────────┘   ← Add（残差接続）: x + F(x)
         │
         ↓
    ┌──────────┐
    │LayerNorm │             ← Norm
    └────┬─────┘
         ↓
       出力
```

#### 数式

```
output = LayerNorm(x + Sublayer(x))
                   ↑
                  Add
```

#### なぜ Add（残差接続）が必要か

| 理由 | 説明 |
|------|------|
| **勾配の流れ** | 深い層でも勾配が消失しにくい（ショートカットパス） |
| **恒等写像** | `F(x) = 0` を学習すれば入力をそのまま出力できる |
| **学習の安定化** | 各層の変化を小さく保てる |

---

## Q22: なぜ「残差」と呼ぶのか？英語では何と表現する？

**質問日**: 2025年12月21日

### 質問

「残差接続」の「残差」とはどういう意味ですか？英語では何と言いますか？

### 回答

#### 英語での表現

**Residual Connection** または **Skip Connection**

#### なぜ「残差」と呼ぶか

元々は **ResNet（Residual Network, 2015）** で導入された概念です。

##### 考え方

通常のネットワークは目標の関数 `H(x)` を直接学習しようとします：

```
出力 = H(x)
```

しかしResNetでは、**入力との差分（残差）** を学習します：

```
F(x) = H(x) - x    ← これが「残差」(Residual)
```

つまり：

```
H(x) = F(x) + x
出力 = F(x) + x
       ↑      ↑
     残差   入力をそのまま加算
```

##### なぜ「残差」なのか

| 用語 | 意味 |
|------|------|
| **Residual** | 「残り」「差分」という意味 |
| **残差** | 目標値と入力の差 = `H(x) - x` |

ネットワークが学習するのは「入力からどれだけ変化させるべきか」という**差分**だけ。
何も変化させる必要がなければ `F(x) = 0` を学習すればよい。

##### 統計学での「残差」

回帰分析でも同じ用語を使います：
```
残差 = 実測値 - 予測値
```

これと同じ発想で、「目標 - 入力 = 残差」を学習するという意味です。

---

## Q23: ResNetとは何か？

**質問日**: 2025年12月21日

### 質問

ResNetとは簡単に言うと何ですか？

### 回答

**2015年にMicrosoftが発表した画像認識のためのディープニューラルネットワーク**

#### 解決した問題

それまでの課題：層を深くすると精度が**下がる**（勾配消失・劣化問題）

```
層を増やす → 精度向上 → ある点から精度低下 😢
```

#### ResNetの解決策

**残差接続（Skip Connection）を導入**

```
従来:    x → [層] → 出力

ResNet:  x → [層] → (+) → 出力
         └─────────↗
            ショートカット
```

#### 結果

| 項目 | 内容 |
|------|------|
| 層数 | 152層まで積み重ね可能に（従来は20層程度が限界） |
| 成果 | ImageNet 2015で優勝、人間の精度を超えた |
| 影響 | Transformerを含む現代のほぼ全てのディープラーニングで採用 |

#### 一言でまとめると

> **「入力をそのまま出力に足す」という単純なアイデアで、超深いネットワークを可能にした**

---

## Q24: Encoderの出力はどういう意味があるのか？Decoderでどう使われるのか？

**質問日**: 2025年12月21日

### 質問

Encoderの出力自体では損失を計算しないので、本当に意味のある学習がなされるのですか？
Decoderでどのように使われますか？

### 回答

#### 1. Encoder出力とは何か

Encoderの出力は、入力シーケンスの**文脈を考慮した表現（Contextual Representation）**です。

```
入力: ["I", "love", "cats"]
        ↓
     Encoder
        ↓
出力: [h₁, h₂, h₃]  ← 各トークンが他のトークンの情報を含む
```

例えば「bank」という単語：
- 「river bank」→ 川岸の意味を反映した表現
- 「money bank」→ 銀行の意味を反映した表現

#### 2. Decoderでの使われ方：Cross-Attention

```
Decoder側
    ↓
┌─────────────────────────────────┐
│  Cross-Attention                │
│  Q = Decoder の状態             │
│  K, V = Encoder の出力 ←────────│── ここで使う！
└─────────────────────────────────┘
```

翻訳タスクの例（英→日）：
```
Encoder入力: "I love cats"
Encoder出力: [h_I, h_love, h_cats]

Decoder生成時:
  "私" を生成 → h_I に強くAttend
  "は" を生成 → h_I に強くAttend  
  "猫" を生成 → h_cats に強くAttend
  "が" を生成 → h_cats に強くAttend
  "好き" を生成 → h_love に強くAttend
```

#### 3. Encoderは本当に学習するのか？

**はい、学習します。** 鍵は**誤差逆伝播（Backpropagation）**です。

```
順伝播:
  入力 → Encoder → Encoder出力 → Decoder → 予測

逆伝播:
  損失 ← Decoder ← Cross-Attention ← Encoder出力 ← Encoder
                         ↑
                    ここを通って勾配がEncoderに流れる
```

Decoderが正しい出力を生成できなければ、**その責任はEncoderにも伝わる**。
Encoderは「Decoderが使いやすい表現」を学習するように最適化されます。

#### 4. 図解

```
        Encoder                     Decoder
    ┌───────────┐              ┌───────────┐
入力→│           │→ Encoder出力 →│           │→ 出力
    │           │      ↓        │Cross-Attn │
    │           │    K, V       │  Q=自身   │
    └───────────┘              └───────────┘
         ↑                           │
         └───────── 勾配 ────────────┘
```

#### まとめ

| 疑問 | 回答 |
|------|------|
| Encoder出力の意味 | 文脈を考慮した入力の表現 |
| Decoderでの使い方 | Cross-AttentionのK, Vとして使用 |
| Encoderは学習するか | はい、Decoderの損失が逆伝播で伝わる |

---

## Q25: Encoderは複数のEncoder Layerが直列に接続されているのか？

**質問日**: 2025年12月21日

### 質問

Encoderは複数個のEncoderが直列に接続されているという理解で良いですか？

### 回答

はい、その理解で正しいです。

#### 構造

```
入力埋め込み + Position Encoding
              ↓
    ┌─────────────────────┐
    │   Encoder Layer 1   │  ← 同じ構造
    └──────────┬──────────┘
              ↓
    ┌─────────────────────┐
    │   Encoder Layer 2   │  ← 同じ構造
    └──────────┬──────────┘
              ↓
             ...
              ↓
    ┌─────────────────────┐
    │   Encoder Layer N   │  ← 同じ構造
    └──────────┬──────────┘
              ↓
          Encoder出力
```

#### 重要なポイント

| 項目 | 内容 |
|------|------|
| 層数 N | 元論文では **N=6** |
| 各層の構造 | 同じ（Self-Attention + FFN + Add & Norm） |
| パラメータ | **各層は独立**（共有しない） |
| 入出力形状 | すべての層で同じ `[batch, seq_len, d_model]` |

#### コードでの実装

```python
# src/encoder.py
self.layers = nn.ModuleList([
    EncoderLayer(d_model, num_heads, d_ff, dropout)
    for _ in range(num_layers)  # N個のLayerを作成
])

# 順伝播
for layer in self.layers:
    x = layer(x, mask)  # 直列に接続
```

#### なぜ複数層を積むのか

1. **浅い層**: 局所的な関係を学習（隣接する単語の関係）
2. **深い層**: 抽象的・大域的な関係を学習（文全体の意味）

---

## Q26: 「文脈を考慮した表現」とは具体的に何を意味するのか？

**質問日**: 2025年12月21日

### 質問

「入力シーケンスの文脈を考慮した表現」とは抽象的ですが、具体的に何を意味しているのですか？

### 回答

#### Before: Token Embedding（文脈なし）

```python
# 単語は常に同じベクトル
embedding["bank"] = [0.2, -0.5, 0.8, ...]  # 常に同じ
```

**問題**: 同じ単語でも意味が違う場合がある
```
"I went to the bank to deposit money"  → bank = 銀行
"I sat on the river bank"              → bank = 川岸
```
でも embedding["bank"] は**同じベクトル**。

---

#### After: Encoder出力（文脈あり）

Encoderを通すと、**周囲の単語の情報が混ざる**。

```
文1: "I went to the bank to deposit money"
                    ↓ Encoder
     bank の表現 = [0.3, -0.2, 0.9, ...]  ← money, deposit の情報を含む

文2: "I sat on the river bank"
                    ↓ Encoder  
     bank の表現 = [-0.1, 0.4, 0.5, ...]  ← river, sat の情報を含む
```

**結果**: 同じ「bank」でも異なるベクトルになる。

---

#### どうやって混ざるのか？

**Self-Attention** が他の単語の情報を集める。

```
"The cat sat on the mat"

"cat" の表現を計算するとき:
  - "The" から少し情報を取る（冠詞）
  - "sat" から多く情報を取る（猫が座っている）
  - "mat" から情報を取る（どこに座っているか）
  
結果: "cat" = 元の埋め込み + 周囲の情報の重み付き和
```

---

#### まとめ

| 段階 | 表現 | 特徴 |
|------|------|------|
| Token Embedding | 静的 | 同じ単語 = 同じベクトル |
| Encoder出力 | 動的 | 同じ単語でも文脈で変わる |

**「文脈を考慮した表現」= 周囲の単語の情報が混ざったベクトル**

---

## Q27: 同じ単語が複数回出現する場合はどう区別されるのか？

**質問日**: 2025年12月21日

### 質問

文章の中に複数の違う意味を持った同じ単語が現れる場合はどうなりますか？シーケンスで見分けるのですか？

### 回答

#### 例文

```
"The bank near the river bank has a bank account service"
     ↑              ↑              ↑
   銀行(1)         川岸           銀行(2)
```

#### 各位置は独立した表現を持つ

Encoderは**トークンの位置ごとに**別々の表現を出力します。

```
位置:    0     1      2     3     4      5     6    7      8       9
入力:  [The] [bank] [near] [the] [river] [bank] [has] [a] [bank] [account]
                ↓ Encoder
出力:  [h₀]  [h₁]   [h₂]  [h₃]   [h₄]   [h₅]  [h₆] [h₇]  [h₈]    [h₉]
              ↑                          ↑                 ↑
         銀行の文脈              川岸の文脈         銀行の文脈
```

**h₁, h₅, h₈ は全て「bank」だが、異なるベクトル**になる。

---

#### なぜ区別できるのか

##### 1. Position Encoding

```
bank(位置1) = embedding["bank"] + PE[1]
bank(位置5) = embedding["bank"] + PE[5]
bank(位置8) = embedding["bank"] + PE[8]
```

位置が違うので、**入力時点で既に異なる**。

##### 2. Self-Attentionで異なる単語に注目

```
bank(位置1) の計算時:
  → "service", "account" に強くAttend → 銀行の意味

bank(位置5) の計算時:
  → "river" に強くAttend → 川岸の意味

bank(位置8) の計算時:
  → "account", "service" に強くAttend → 銀行の意味
```

---

#### 図解

```
入力シーケンス:
  位置:  1       5       8
       [bank]  [bank]  [bank]
         ↓       ↓       ↓
       +PE[1]  +PE[5]  +PE[8]   ← 位置情報で区別
         ↓       ↓       ↓
    ┌─────────────────────────┐
    │    Self-Attention       │  ← 周囲の単語を見て
    │  各位置が異なる単語に注目   │     文脈を取り込む
    └─────────────────────────┘
         ↓       ↓       ↓
       [h₁]    [h₅]    [h₈]    ← 全て異なるベクトル
       銀行    川岸     銀行
```

---

#### まとめ

| 仕組み | 役割 |
|--------|------|
| **Position Encoding** | 同じ単語でも位置で区別 |
| **Self-Attention** | 周囲の単語に基づき意味を決定 |
| **出力形状** | `[seq_len, d_model]` = 各位置に1つのベクトル |

**シーケンスの各位置は常に独立した表現を持つ**ので、同じ単語が何回出ても混同しない。

---

## Q28: 複数の特徴量ベクトルが一つの単語に結びつくのか？

**質問日**: 2025年12月21日

### 質問

人間が理解できる文章を生成する際に特徴量ベクトルと単語の辞書を引くと理解していますが、複数の特徴量ベクトルに一つの単語が結びつくのですか？

### 回答

はい、その理解で正しいです。

#### Decoderの出力層

```
Decoder出力: [h₁, h₂, h₃, ...]   (各位置のベクトル, d_model次元)
                 ↓
            線形変換 (d_model → vocab_size)
                 ↓
            Softmax
                 ↓
           確率分布: [P("the"), P("cat"), P("sat"), ...]
                 ↓
            argmax（最大確率の単語を選択）
                 ↓
              出力単語
```

#### 複数のベクトル → 同じ単語

**はい、異なるベクトルが同じ単語に変換されます。**

```
文1: "I love cats"
     Decoder出力 h₁ = [0.8, 0.3, -0.2, ...] → "I"

文2: "I hate dogs"  
     Decoder出力 h₁ = [0.7, 0.4, -0.1, ...] → "I"
     
     ベクトルは違うが、同じ "I" を出力
```

#### なぜベクトルが違うのか

ベクトルには**次に何が来るかの情報**も含まれている。

```
"I love ..." の "I" のベクトル:
  → "love" や "cats" に繋がる方向を向いている

"I hate ..." の "I" のベクトル:
  → "hate" や "dogs" に繋がる方向を向いている
```

#### 図解

```
            特徴量空間                    語彙空間
         (d_model次元)                (vocab_size次元)
         
    h₁ ●──────┐
              │
    h₂ ●──────┼─────→  "the" ●
              │
    h₃ ●──────┘
    
    h₄ ●─────────────→  "cat" ●
    
    h₅ ●──────┐
              ├─────→  "is"  ●
    h₆ ●──────┘
```

**多対一の関係**: 複数の特徴量ベクトルが1つの単語に写像される。

#### コードでの実装

```python
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model):
        # 出力層: d_model → vocab_size
        self.output_projection = nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        # x: [batch, seq_len, d_model]
        logits = self.output_projection(x)  # [batch, seq_len, vocab_size]
        probs = F.softmax(logits, dim=-1)
        return probs
```

#### まとめ

| 質問 | 回答 |
|------|------|
| 複数ベクトル → 1単語？ | はい、多対一の写像 |
| ベクトルの違いは何？ | 文脈情報（前後の単語との関係） |
| どうやって単語に変換？ | 線形変換 + Softmax + argmax |

---

## Q29: 翻訳タスクでのEncoder/Decoderの入出力は？

**質問日**: 2025年12月22日

### 質問

日英翻訳タスクの場合、Encoderの入力は日本語シーケンス、Decoderへの入力はその訳である英文シーケンスという理解で良いですか？

### 回答

はい、その理解で正しいです。

```
日本語: "私は猫が好きです"
         ↓
    ┌─────────┐
    │ Encoder │  ← 日本語シーケンス（ソース）
    └────┬────┘
         ↓ Encoder出力
    ┌─────────┐
    │ Decoder │  ← 英語シーケンス（ターゲット）
    └────┬────┘
         ↓
英語: "I love cats"
```

#### 学習時 vs 推論時

| フェーズ | Encoder入力 | Decoder入力 | Decoder出力 |
|---------|------------|-------------|-------------|
| **学習時** | 日本語文 | 正解の英語文（シフト） | 次のトークン予測 |
| **推論時** | 日本語文 | 生成済みトークン | 次のトークン |

#### 学習時の詳細（Teacher Forcing）

```
Encoder入力: ["私", "は", "猫", "が", "好き", "です"]

Decoder入力: ["<start>", "I", "love", "cats"]  ← 正解を1つずらして入力
Decoder出力: ["I", "love", "cats", "<end>"]    ← 次のトークンを予測
```

#### 推論時の詳細（自己回帰生成）

```
Step 1: Decoder入力 ["<start>"]        → 予測 "I"
Step 2: Decoder入力 ["<start>", "I"]   → 予測 "love"
Step 3: Decoder入力 ["<start>", "I", "love"] → 予測 "cats"
Step 4: Decoder入力 ["<start>", "I", "love", "cats"] → 予測 "<end>"
```

---

## Q30: Decoderの学習でTeacher Forcingとは何か？

**質問日**: 2025年12月22日

### 質問

Decoderの出力は次のトークンの予測で、それをDecoder入力として使い、答えの英文は教師信号として使うという理解で良いですか？

### 回答

はい、正確な理解です。

#### 学習時の流れ

```
正解英文: ["I", "love", "cats", "<end>"]
              ↑
          教師信号（損失計算に使用）

Decoder入力: ["<start>", "I", "love", "cats"]  ← Teacher Forcing
                  ↓
             ┌─────────┐
             │ Decoder │
             └────┬────┘
                  ↓
Decoder出力: [P("I"), P("love"), P("cats"), P("<end>")]
                  ↓
              損失計算（Cross-Entropy Loss）
                  ↓
            正解と比較して学習
```

#### ポイント

| 項目 | 内容 |
|------|------|
| **Decoder入力** | 正解文を1つ右にシフト（Teacher Forcing） |
| **Decoder出力** | 各位置で次トークンの確率分布 |
| **教師信号** | 正解の英文（シフトなし） |
| **損失関数** | 出力確率と正解トークンのCross-Entropy |

#### なぜTeacher Forcingを使うか

```
❌ 自分の予測を入力に使う場合（推論時と同じ）:
  間違った予測 → 次も間違える → エラーが蓄積

✅ Teacher Forcing（正解を入力に使う）:
  常に正解を入力 → 安定した学習 → 高速収束
```

#### 推論時との違い

| フェーズ | Decoder入力 | 
|---------|------------|
| **学習時** | 正解文（Teacher Forcing） |
| **推論時** | 自分の予測を使う（自己回帰） |

---

## Q31: 翻訳時にソースとターゲットのシーケンス長が異なる場合はどこで吸収するのか？

**質問日**: 2025年12月22日

### 質問

日英翻訳の場合、日本語と英語でシーケンス長に違いが生じますが、どこで吸収していますか？

### 回答

**Cross-Attention**で吸収します。

#### Cross-Attentionの行列サイズ

```
日本語（ソース）: 5トークン (src_len = 5)
英語（ターゲット）: 8トークン (tgt_len = 8)

Cross-Attention:
  Q = Decoder状態     [tgt_len, d_model] = [8, 512]
  K = Encoder出力     [src_len, d_model] = [5, 512]
  V = Encoder出力     [src_len, d_model] = [5, 512]
```

#### 行列積の計算（なぜ次元が合うのか）

```
K^T（Kの転置）= [d_model, src_len] = [512, 5]
                 ↑ 行と列が入れ替わる

Q @ K^T の計算:
  [tgt_len, d_model] @ [d_model, src_len]
  [8,       512    ] @ [512,     5      ]
       ↑                   ↑
       └─── 共通次元（512）で内積 ───┘
              ↓
         結果: [8, 5]
```

**d_model次元が共通**なので、tgt_lenとsrc_lenが異なっても計算可能です。

#### 図解

```
        Q [8, 512]              K^T [512, 5]
     ┌─────────────┐          ┌───────────┐
     │ → → → → → → │ 512次元  │ ↓ ↓ ↓ ↓ ↓ │
  8行│ → → → → → → │    @     │ ↓ ↓ ↓ ↓ ↓ │ 5列
     │ → → → → → → │          │ ↓ ↓ ↓ ↓ ↓ │ 512行
     │    ...      │          │   ...     │
     └─────────────┘          └───────────┘
           ↓
      結果 [8, 5]
     ┌───────────┐
     │ ● ● ● ● ● │  ← 各要素は512次元の内積
  8行│ ● ● ● ● ● │    = Qの各行とKの各行の類似度
     │   ...     │
     └───────────┘
        5列
```

#### 完全な計算の流れ

```
Attention重み = softmax(Q @ K^T / √d_k)
             = softmax([8, 512] @ [512, 5] / √512)
             = softmax([8, 5])
             = [8, 5]  ← 各行の和が1になる

出力 = Attention重み @ V
     = [8, 5] @ [5, 512]
     = [8, 512]  ← ターゲット長に揃う
```

#### 具体例

```
日本語: ["私", "は", "猫", "が", "好き"]  (5トークン)
英語:   ["I", "love", "cats", "very", "much", ".", "<pad>", "<pad>"]  (8トークン)

Cross-Attention重み (8×5):
        私    は    猫    が    好き
"I"    [0.7  0.2  0.0  0.0  0.1 ]  ← 「私」に強く注目
"love" [0.1  0.1  0.1  0.1  0.6 ]  ← 「好き」に強く注目
"cats" [0.0  0.0  0.8  0.1  0.1 ]  ← 「猫」に強く注目
"very" [0.1  0.1  0.1  0.1  0.6 ]  ← 「好き」に注目
"much" [0.1  0.1  0.1  0.1  0.6 ]  ← 「好き」に注目
"."    [0.1  0.2  0.2  0.2  0.3 ]  ← 分散
...
```

各ターゲット位置は、ソース全体（日本語5トークン）への重み付き和を計算し、
結果としてターゲット長（英語8トークン）に揃った表現を得ます。

#### ポイント

| 項目 | 内容 |
|------|------|
| **どこで吸収** | Cross-Attention |
| **なぜ計算可能** | d_model次元が共通なので、Q @ K^T が計算できる |
| **仕組み** | Q [tgt, d] × K^T [d, src] → 重み [tgt, src] |
| **結果** | ターゲットの各位置が、ソース全体を参照できる |

#### Self-Attentionとの違い

| Attention | Q, K, V | 結果のサイズ |
|-----------|---------|-------------|
| **Self-Attention** | 全て同じシーケンス | [seq_len, seq_len] → [seq_len, d_model] |
| **Cross-Attention** | QとK,Vが異なる | [tgt_len, src_len] → [tgt_len, d_model] |